# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 12 2023 11:57AM,254910,12,HH8748,Hush Hush,Released
1,Jan 12 2023 11:52AM,254909,10,0086295945,ISDIN Corporation,Released
2,Jan 12 2023 11:52AM,254909,10,0086295962,ISDIN Corporation,Released
3,Jan 12 2023 11:52AM,254909,10,0086295965,ISDIN Corporation,Released
4,Jan 12 2023 11:52AM,254909,10,0086295966,ISDIN Corporation,Released
5,Jan 12 2023 11:40AM,254907,10,9018699,Eywa Pharma Inc.,Released
6,Jan 12 2023 11:40AM,254907,10,9018761,Eywa Pharma Inc.,Released
7,Jan 12 2023 11:40AM,254907,10,9018766,Eywa Pharma Inc.,Released
8,Jan 12 2023 11:40AM,254907,10,9018768,Eywa Pharma Inc.,Released
9,Jan 12 2023 11:40AM,254907,10,9018770,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,254905,Released,1
33,254907,Released,14
34,254908,Released,2
35,254909,Released,4
36,254910,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254905,NaN,NaN,1.0
254907,NaN,NaN,14.0
254908,NaN,NaN,2.0
254909,NaN,NaN,4.0
254910,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254806,0.0,0.0,1.0
254810,0.0,1.0,0.0
254811,0.0,1.0,0.0
254813,0.0,0.0,1.0
254814,33.0,0.0,20.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254806,0,0,1
254810,0,1,0
254811,0,1,0
254813,0,0,1
254814,33,0,20


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254806,0,0,1
1,254810,0,1,0
2,254811,0,1,0
3,254813,0,0,1
4,254814,33,0,20


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254806,,,1
1,254810,,1,
2,254811,,1,
3,254813,,,1
4,254814,33,,20


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 12 2023 11:57AM,254910,12,Hush Hush
1,Jan 12 2023 11:52AM,254909,10,ISDIN Corporation
5,Jan 12 2023 11:40AM,254907,10,Eywa Pharma Inc.
19,Jan 12 2023 11:39AM,254908,19,"GUSA Granules USA, Inc."
21,Jan 12 2023 11:37AM,254904,10,Bio-PRF
34,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc."
35,Jan 12 2023 10:38AM,254901,10,Methapharm-G
37,Jan 12 2023 10:38AM,254901,10,"Methapharm, Inc."
39,Jan 12 2023 10:20AM,254899,17,"Citieffe, Inc."
40,Jan 12 2023 9:44AM,254894,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 12 2023 11:57AM,254910,12,Hush Hush,,,1
1,Jan 12 2023 11:52AM,254909,10,ISDIN Corporation,,,4
2,Jan 12 2023 11:40AM,254907,10,Eywa Pharma Inc.,,,14
3,Jan 12 2023 11:39AM,254908,19,"GUSA Granules USA, Inc.",,,2
4,Jan 12 2023 11:37AM,254904,10,Bio-PRF,,,13
5,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc.",,,1
6,Jan 12 2023 10:38AM,254901,10,Methapharm-G,,2,2
7,Jan 12 2023 10:38AM,254901,10,"Methapharm, Inc.",,2,2
8,Jan 12 2023 10:20AM,254899,17,"Citieffe, Inc.",,,1
9,Jan 12 2023 9:44AM,254894,10,ISDIN Corporation,,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 11:57AM,254910,12,Hush Hush,1,,
1,Jan 12 2023 11:52AM,254909,10,ISDIN Corporation,4,,
2,Jan 12 2023 11:40AM,254907,10,Eywa Pharma Inc.,14,,
3,Jan 12 2023 11:39AM,254908,19,"GUSA Granules USA, Inc.",2,,
4,Jan 12 2023 11:37AM,254904,10,Bio-PRF,13,,
5,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc.",1,,
6,Jan 12 2023 10:38AM,254901,10,Methapharm-G,2,2,
7,Jan 12 2023 10:38AM,254901,10,"Methapharm, Inc.",2,2,
8,Jan 12 2023 10:20AM,254899,17,"Citieffe, Inc.",1,,
9,Jan 12 2023 9:44AM,254894,10,ISDIN Corporation,5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 11:57AM,254910,12,Hush Hush,1,,
1,Jan 12 2023 11:52AM,254909,10,ISDIN Corporation,4,,
2,Jan 12 2023 11:40AM,254907,10,Eywa Pharma Inc.,14,,
3,Jan 12 2023 11:39AM,254908,19,"GUSA Granules USA, Inc.",2,,
4,Jan 12 2023 11:37AM,254904,10,Bio-PRF,13,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 11:57AM,254910,12,Hush Hush,1.0,NaN,NaN
1,Jan 12 2023 11:52AM,254909,10,ISDIN Corporation,4.0,NaN,NaN
2,Jan 12 2023 11:40AM,254907,10,Eywa Pharma Inc.,14.0,NaN,NaN
3,Jan 12 2023 11:39AM,254908,19,"GUSA Granules USA, Inc.",2.0,NaN,NaN
4,Jan 12 2023 11:37AM,254904,10,Bio-PRF,13.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 11:57AM,254910,12,Hush Hush,1.0,0.0,0.0
1,Jan 12 2023 11:52AM,254909,10,ISDIN Corporation,4.0,0.0,0.0
2,Jan 12 2023 11:40AM,254907,10,Eywa Pharma Inc.,14.0,0.0,0.0
3,Jan 12 2023 11:39AM,254908,19,"GUSA Granules USA, Inc.",2.0,0.0,0.0
4,Jan 12 2023 11:37AM,254904,10,Bio-PRF,13.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1784309,42.0,6.0,0.0
12,254910,1.0,0.0,0.0
15,764545,33.0,72.0,43.0
16,254874,1.0,0.0,0.0
17,254899,1.0,0.0,0.0
19,1019544,24.0,39.0,3.0
21,3312743,10.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1784309,42.0,6.0,0.0
1,12,254910,1.0,0.0,0.0
2,15,764545,33.0,72.0,43.0
3,16,254874,1.0,0.0,0.0
4,17,254899,1.0,0.0,0.0
5,19,1019544,24.0,39.0,3.0
6,21,3312743,10.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,42.0,6.0,0.0
1,12,1.0,0.0,0.0
2,15,33.0,72.0,43.0
3,16,1.0,0.0,0.0
4,17,1.0,0.0,0.0
5,19,24.0,39.0,3.0
6,21,10.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,42.0
1,12,Released,1.0
2,15,Released,33.0
3,16,Released,1.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,19,21
Status,,,,,,,
Completed,0.0,0.0,43.0,0.0,0.0,3.0,0.0
Executing,6.0,0.0,72.0,0.0,0.0,39.0,3.0
Released,42.0,1.0,33.0,1.0,1.0,24.0,10.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,19,21
0,Completed,0.0,0.0,43.0,0.0,0.0,3.0,0.0
1,Executing,6.0,0.0,72.0,0.0,0.0,39.0,3.0
2,Released,42.0,1.0,33.0,1.0,1.0,24.0,10.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,19,21
0,Completed,0.0,0.0,43.0,0.0,0.0,3.0,0.0
1,Executing,6.0,0.0,72.0,0.0,0.0,39.0,3.0
2,Released,42.0,1.0,33.0,1.0,1.0,24.0,10.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()